In [70]:
import json
import pandas as pd

In [71]:
def format_number(x):
    if isinstance(x, float):
        if x.is_integer():
            return int(x)  # Convert to integer if the float is a whole number
        else:
            return '{:.2f}'.format(x).rstrip('0').rstrip('.')  # Keep up to 2 decimals, remove trailing zeros
    return x

def flatten_json(json_data):
    data = []
    for circuit, circuit_value in json_data.items(): # circuit size
        for benchmark, benchmark_value in circuit_value.items(): # benchmark_name
            for layout, metrics in benchmark_value.items():
                if layout == 'init':
                    continue
                if 'lookahead_swap' not in metrics:
                    continue
                basic_swap = metrics.get('basic_swap', None)
                sabre_swap = metrics.get('sabre_swap', None)
                lookahead_swap = metrics.get('lookahead_swap', None)
                swap_change = ((lookahead_swap - sabre_swap) / sabre_swap * 100) if sabre_swap != 0 and sabre_swap is not None and lookahead_swap is not None else None
                
                basic_depth = metrics.get('basic_depth', None)
                sabre_depth = metrics.get('sabre_depth', None)
                lookahead_depth = metrics.get('lookahead_depth', None)
                depth_change = ((lookahead_depth - sabre_depth) / sabre_depth * 100) if sabre_depth != 0 and sabre_depth is not None and lookahead_depth is not None else None

                data.append({
                    "Benchmark": benchmark,
                    "Size": circuit, # circuit size
                    "Gate": benchmark_value['init']['size'], # total gates
                    "Depth": benchmark_value['init']['depth'], # circuit depth
                    "Layout": layout,
                    "Swap Basic": basic_swap, # basic swap
                    "Swap Sabre": sabre_swap, # sabre swap
                    "Swap Lookahead": lookahead_swap, # lookahead swap
                    "Swap Percentage": swap_change,
                    "Depth Basic": basic_depth, # basic depth
                    "Depth Sabre": sabre_depth, # sabre depth
                    "Depth Lookahead": lookahead_depth, # lookahead depth
                    "Depth Percentage": depth_change
                })
    return pd.DataFrame(data)


In [72]:
def open_json_from_file():
    with open("result/benchmarking_2.json") as json_file:
        return json.load(json_file)

dict_benchmark = open_json_from_file()
print(len(dict_benchmark["5"].keys()))

15


In [73]:
a = dict_benchmark["5"]["ghz"]["full_10_2"]
a_sabre_swap = a.get("sabre_swap", None)
print(isinstance(a_sabre_swap, int))

True


In [74]:
df = flatten_json(dict_benchmark)
df = df.map(format_number)
df.columns = [
    "Benchmark",
    "n",
    "g",
    "d",
    "layout",
    "s basic", # basic swap
    "s sabre", # sabre swap
    "s look", # lookahead swap
    "swap (%)",
    "d basic", # basic depth
    "d swap", # sabre depth
    "d look", # lookahead depth
    "d (%)",
]
# df.columns = [
#     "Benchmark",
#     "n",
#     "g",
#     "d",
#     "layout",
#     r"s\_B", # basic swap
#     r"s\_S", # sabre swap
#     r"s\_L", # lookahead swap
#     r"s (%)",
#     r"d_B", # basic depth
#     r"d_S", # sabre depth
#     r"d_L", # lookahead depth
#     r"d (%)",
# ]


print(df)

    Benchmark   n    g    d            layout  s basic  s sabre  s look  \
0          dj   5   36   11         full_10_2        0        0       0   
1          dj   5   36   11          full_7_3        0        3       0   
2          dj   5   36   11         ring_10_2       36        3       3   
3          dj   5   36   11          ring_7_3       24        3       3   
4          dj   5   36   11          grid_9_3        9        3       0   
..        ...  ..  ...  ...               ...      ...      ...     ...   
380    wstate  15  253  135  t_horizontal_5_4       63       21      45   
381    wstate  15  253  135    t_vertical_5_4      126       45      84   
382    wstate  15  253  135          ring_7_3        0       15     108   
383    wstate  15  253  135          full_7_3        0       12       0   
384    wstate  15  253  135          ring_5_4        0       48      72   

    swap (%)  d basic  d swap  d look   d (%)  
0        nan       11      11      11       0  
1  

In [75]:
def save_latex_to_file(latex, circuit_size):
    with open(f"result/table_{circuit_size}.tex", "w") as file:
        file.write(latex)

In [76]:
df_n_5 = df[df["n"] == "5"]
# df_n_5 = df_n_5.replace('_', r'\_', regex=True)
df_n_5 = df_n_5.drop(columns=["n"])
latex_n_5 = df_n_5.to_latex(index=False, escape=False, longtable=True)
latex_n_5 = latex_n_5.replace('_', r'\_').replace('%', r'\%')
# latex_n_5 = latex_n_5.replace('%', r'\%')
save_latex_to_file(latex_n_5, 5)

In [77]:
df_n_10 = df[df["n"] == "10"]
df_n_10 = df_n_10.drop(columns=["n"])
latex_n_10 = df_n_10.to_latex(index=False, escape=False, longtable=True)
latex_n_10 = latex_n_10.replace('_', r'\_').replace('%', r'\%')
save_latex_to_file(latex_n_10, 10)

In [78]:
df_n_15 = df[df["n"] == "15"]
df_n_15 = df_n_15.drop(columns=["n"])
latex_n_15 = df_n_15.to_latex(index=False, escape=False, longtable=True)
latex_n_15 = latex_n_15.replace('_', r'\_').replace('%', r'\%')
save_latex_to_file(latex_n_15, 15)